<a href="https://colab.research.google.com/github/MatinDelkhosh/hello-world/blob/master/SngthMat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sympy import *
from sympy.physics.units.quantities import Quantity as Q
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display,Latex

def beauti(x): display(Latex(latex(x)))

Sf = SingularityFunction
x = Symbol('x')
S = Symbol
intg = integrate

def Constant_w(w,L,a,b,c):
  if a > L or a > b: print('wrong input')
  else: w += c*(Sf(x,a,0) - Sf(x,b,0))
  return w

def Point_Force(w,L,l,F):
  if l > L: print('wrong input')
  else : w -= F*(Sf(x,l,-1))
  return w

def Point_Torque(w,L,l,T):
  if l > L: print('wrong input')
  else: w -= T*(Sf(x,l,-2))
  return w

def Sloped_w(w,L,a,b,c,d):
  if a > L or a > b: print('wrong input'); return w
  m = (d-c)/(b-a)
  w = Constant_w(w,L,a,b,c)
  w += m*(Sf(x,a,1) - Sf(x,b,1)) - (d-c)*(Sf(x,b,0))
  return w

def support(L,init,supps):
  try:
    loc = float(input('location: '))
    b_type = input('what type of supporting h/hinge w/weld s/stand f/free: ')
    match b_type:
      case 'h':
        init['y'].append((loc, 0))
        supps['Hinge'].append(loc)
      case 'w':
        if loc == 0 or loc == L:
          init['y'].append((loc, 0))
          init['y_p'].append((loc, 0))
          supps['weld'].append(loc)
        else: print('wrong location for weld')
      case 's':
        init['y'].append((loc, 0))
        supps['stand'].append(loc)
      case 'f':
        if loc == 0 or loc == L:
          supps['Free'].append(loc)
  except: pass

def Distribution(T = True):
  print('commands: w/constant weight s/sloped weight f/Force t/Torque b/support')
  while True:
    try: L = float(input('length of the bar: ')); break
    except: continue
  w = 0
  init = {'y_p':[],'y':[]}
  supps = {'Hinge':[],'weld':[],'stand':[],'Free':[]}
  EI = float(input('EI: '))
  while T:
    cmd = input('what to add: ')
    if cmd == 'w':
      try:
        a = float(input('Start: '))
        b = float(input('End: '))
        c = float(input('amount: '))
      except: print('wrong input'); continue
      w = Constant_w(w,L,a,b,c)

    elif cmd == 's':
      try:
        a = float(input('Start: '))
        b = float(input('End: '))
        c = float(input('start amount: '))
        d = float(input('end amount: '))
      except: print('wrong input'); continue
      w = Sloped_w(w,L,a,b,c,d)

    elif cmd == 'f':
      try:
        l = float(input('location: '))
        F = float(input('amount: '))
      except: print('wrong input'); continue
      w = Point_Force(w,L,l,F)

    elif cmd == 't':
      try:
        l = float(input('location: '))
        T = float(input('amount: '))
      except: print('wrong input'); continue
      w = Point_Torque(w,L,l,T)

    elif cmd == 'b':
      support(L,init,supps)

    elif cmd == 'done':
      return (w,L,supps,init,EI)
    beauti(w)

def static(w,L,supps,init,EI):
  Fx,Fy,M = [],[],[]
  for i in supps['Hinge']:
    Fx.append(S(f'Fx{len(Fx)+1}'))
    Fy.append((S(f'Fy{len(Fy)+1}'),i))
  for i in supps['weld']:
    Fx.append(S(f'Fx{len(Fx)+1}'))
    Fy.append((S(f'Fy{len(Fy)+1}'),i))
    M.append((S(f'M{len(M)+1}'),i))
  for i in supps['stand']:
    Fy.append((S(f'Fy{len(Fy)+1}'),i))
  Eqfx = Eq(sum(Fx),0)
  Eqfy = Eq(sum(i[0] for i in Fy) - intg(w).subs(x,L),0)
  EqM  = Eq(sum(i[0]*i[1] for i in Fy) + sum(i[0] for i in M) - intg(intg(w)).subs(x,L), 0)
  Fxs = solve([Eqfx],Fx)
  FyM = solve([Eqfy,EqM],list(i[0] for i in Fy)+list(i[0] for i in M))
  for i in Fy:
    w = Point_Force(w,L,i[1],-i[0])
    print(f'added {i}');beauti(w)
  for i in M:
    w = Point_Torque(w,L,i[1],i[0])
    print(f'added {i}');beauti(w)
  for i in FyM.keys():
    w = w.subs(i,FyM[i])
  N = []; print(FyM)
  if len(M+Fy) > len(FyM):
    N = list(list(i) for i in M+Fy if i[0] not in FyM.keys())
  return (w,EI,init,L,N,supps)

def mbracket(w,EI,init,L,N,supps):
  print('w is:');  beauti(w)
  c1, c2 = Q('c1'),Q('c2')
  w = str(w)
  for i in N: locals()[str(i[0])] = Q(str(i[0]))
  w = eval(w)
  V = intg(w) ; beauti(V)
  M = intg(V)
  y_p = (intg(M) + c1)/EI
  y = intg(y_p) + c2
  V = str(V); M = str(M)
  y_p = str(y_p); y = str(y)
  for i in N: locals()[str(i[0])] = S(str(i[0]))
  c1, c2 = S('c1'),S('c2')
  V = eval(V); M = eval(M)
  y_p = eval(y_p); y = eval(y)
  eqs = []; cs = [c1,c2] + list(i[0] for i in N)
  comp = sum(len(i) for i in init.values()) - len(cs)
  if comp > 0:
    eqs.append(Eq(((y_p*x)-y).subs(x,supps['weld'][0]),0))
  for i in init['y_p']:
    eqs.append(Eq(y_p.subs(x,i[0]),i[1]))
  for i in init['y']:
    eqs.append(Eq(y.subs(x,i[0]),i[1]))
  inits = solve(eqs,cs); print('variables: ',cs, '\nand the Eqs are:')
  for i in eqs: beauti(i)
  print(inits)
  if type(inits) is list:
    inits = dict((j,i) for i in inits[0] for j in cs)
  for i in inits.keys():
    y_p = y_p.subs(i,inits[i])
    y = y.subs(i,inits[i])
  return [w,V,M,y_p,y,L]

data = mbracket(*static(*Distribution()))
i = int(input("what to plot? w/0 V/1 M/2 y'/3 y/4\n : "))
beauti(data[i])
p1 = plot(data[i],show = False,xlim=[0,data[-1]],ylim=[-20,20])
p1.show()